In [ ]:
pip install torch torchvision torchaudio transformers


In [ ]:
import os
import time
import torch
from torch.quantization import quantize_dynamic
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
nome_modelo = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
modelo = AutoModelForSequenceClassification.from_pretrained(nome_modelo)
modelo.eval()


In [ ]:
texto = "Eu amo a Universidade Federal Fluminense!"
inputs = tokenizer(texto, return_tensors="pt")

with torch.no_grad():
    outputs = modelo(**inputs)

probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
print("Probabilidades (modelo original):", probs.numpy())


In [ ]:
modelo_quantizado = quantize_dynamic(
    modelo,
    {torch.nn.Linear},
    dtype=torch.qint8
)


In [ ]:
with torch.no_grad():
    outputs_q = modelo_quantizado(**inputs)

probs_q = torch.nn.functional.softmax(outputs_q.logits, dim=-1)
print("Probabilidades (modelo quantizado):", probs_q.numpy())


In [ ]:
torch.save(modelo.state_dict(), "original.pt")
torch.save(modelo_quantizado.state_dict(), "quantizado.pt")

tamanho_original = os.path.getsize("original.pt") / 1e6
tamanho_quantizado = os.path.getsize("quantizado.pt") / 1e6

print(f"Tamanho original: {tamanho_original:.2f} MB")
print(f"Tamanho quantizado: {tamanho_quantizado:.2f} MB")


In [ ]:
texto = "Texto para teste de performance sobre os benefícios da quantização."
inputs = tokenizer(texto, return_tensors="pt")

start_time = time.time()
for _ in range(100):
    with torch.no_grad():
        modelo(**inputs)
tempo_original = (time.time() - start_time) / 100

start_time = time.time()
for _ in range(100):
    with torch.no_grad():
        modelo_quantizado(**inputs)
tempo_quantizado = (time.time() - start_time) / 100

print(f"\nTempo de inferência original (média de 100 execuções): {tempo_original:.6f} segundos")
print(f"Tempo de inferência quantizado (média de 100 execuções): {tempo_quantizado:.6f} segundos")

speedup = tempo_original / tempo_quantizado
print(f"O modelo quantizado é {speedup:.2f}x mais rápido.")


In [ ]:
print("\n--- Comparando a Saída dos Modelos ---")

with torch.no_grad():
    probs_original = torch.nn.functional.softmax(modelo(**inputs).logits, dim=-1)
    probs_quantizado = torch.nn.functional.softmax(modelo_quantizado(**inputs).logits, dim=-1)

print("Original:", probs_original.numpy())
print("Quantizado:", probs_quantizado.numpy())

diff = torch.abs(probs_original - probs_quantizado).sum().item()
print(f"Erro absoluto total: {diff:.6f}")
